# Result data analysis and chemical space check

This notebook analyzes the prediciton result from a machine learning model. A cut off value is determined by the user to characterize how much error is acceptable and unacceptable. The the problematic compounds are checked to see if there are patterns, so we can improve the model.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import molplotly
import umap
import bamboolib as bam
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem import Draw

## Input area

In [ ]:
data_path = 'your/data/path.csv'
model_name = 'your_model.pkl'
cut_off = 1.0

data = pd.read_csv(data_path)
model = joblib.load(model_name)

## extract wrongly predicted molecules

In [ ]:
# Generate data set with smiles
X_smi = data.drop(['solubility','sol_M','sol_log_M'], axis = 1)
X_train_smi, X_test_smi = train_test_split(X_smi,test_size=0.2, random_state=42)

In [ ]:
# Genereate dataset that can be used as input to the model
X = data.drop(['smiles','sol_M','sol_log_M'], axis = 1)
y = data['sol_log_M']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

std_scaler_x = StandardScaler()

train_x = std_scaler_x.fit_transform(X_train)
train_y = y_train.ravel()

test_x = std_scaler_x.transform(X_test)
test_y = y_test.ravel()

In [ ]:
# Some functions for metric and percentage calculation
def metric(y_pred,y):
    rmse = mean_squared_error(y, y_pred ,squared=False)
    r_2 = r2_score(y,y_pred)
    return rmse, r_2

def percentage(analysis):
    analysis['difference'] = np.abs(analysis['sol_pred'] - analysis['sol_log_M'])
    analysis_error = analysis.loc[analysis['difference'] > cut_off]
    ratio = len(analysis_error)/len(analysis)
    return ratio, analysis_error

In [ ]:
# train evaluation
train_predictions = model.predict(train_x)
rmse_train, r2_train = metric(train_predictions, train_y)
print('train rmse :', rmse_train,'\n', 'train r^2 :', r2_train)

# test evaluation
test_predictions = model.predict(test_x)
rmse_test, r2_test = metric(test_predictions,test_y)
print('test rmse :', rmse_test,'\n', 'test r^2 :', r2_test)

In [ ]:
# Check how many compounds in the dataset are exceeding the cut off and extract them
error_analysis_train = pd.concat([X_train_smi,y_train],axis = 1)
error_analysis_train['sol_pred'] = train_predictions
log_percentage_train, error_data_train = percentage(error_analysis_train)
print(f'train LogS ± {cut_off}%:', log_percentage_train)

error_analysis_test = pd.concat([X_test_smi,y_test],axis = 1)
error_analysis_test['sol_pred'] = test_predictions
log_percentage_test, error_data_test = percentage(error_analysis_test)
print(f'test LogS ± {cut_off}%:', log_percentage_test)

## Check the structure of the wrongly predicted compounds

In [ ]:
smi = error_data_test['smiles'].to_list()
smi =  [Chem.MolFromSmiles(compound) for compound in smi]
img=Draw.MolsToGridImage(smi,molsPerRow=4,subImgSize=(200,200))  
img

## Check the chemical space with UMAP

In [ ]:
# Prepare dataset for UMAP, mark the categories
error = error_data_test
error = error[['smiles', 'sol_log_M', 'sol_pred', 'difference']]
error['category'] = f'diff > {cut_off}'

accept = error_analysis_test
accept = accept[['smiles', 'sol_log_M', 'sol_pred', 'difference']]
accept = accept.loc[~accept['smiles'].isin(error['smiles'])]
accept['category'] = f'diff < {cut_off}'

sol = pd.concat([accept, error])

In [ ]:
sol['difference'] =sol['difference'].astype(str)

In [ ]:
# Caculate ECFP fingerprints 2018 bits and 3 radius
def smi_to_fp(smi):
    fp = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi), 3, nBits=2048)
    arr = np.zeros((0,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

fps = np.array([smi_to_fp(smi) for smi in sol['smiles']])

In [ ]:
umap_model = umap.UMAP(metric = "jaccard",
                      n_neighbors = 10,
                      n_components = 2,
                      low_memory = False,
                      min_dist = 0.001)
X_umap = umap_model.fit_transform(fps.reshape(-1, 2048))

sol['UMAP-1'] = X_umap[:, 0]
sol['UMAP-2'] = X_umap[:, 1]

In [ ]:
fig_umap = px.scatter(sol,
                     x="UMAP-1",
                     y="UMAP-2",
                     title='Sol UMAP of morgan fingerprints',
                     color='category',
                     width=1200,
                     height=800)

In [ ]:
app_umap = molplotly.add_molecules(fig=fig_umap,
                                  df=sol,
                                  smiles_col='smiles',
                                  title_col = 'difference',
                                  color_col='category') # Note that title_col type cannot be int64, it can only be str

In [ ]:
app_umap.run_server(mode='inline', port=8009, height=800) # Note the port cannot be used twice